In [4]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [5]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [6]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 86
Current browser version is 89.0.4389.90 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome


In [4]:
url = 'http://www.dpdconnect.com/dailybulletin.aspx'
browser.visit(url)

NameError: name 'browser' is not defined

In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mainTable = soup.find('div', id='gbox_tblDB')
listTable= soup.find('div', id="tblDB")
dateField = browser.find_by_id("mainContent_txtDate2")
dateSubmit = browser.find_by_id("mainContent_lbUpdate")

NameError: name 'browser' is not defined

In [ ]:
dateNow=dateField.value
print(dateNow)
dateField.fill('1/1/2021')
dateSubmit.click()
print(dateNow)